# Test model submission workflow

In [1]:
import os
import sys
import re
import requests
import subprocess
import pandas as pd
from IPython.display import Markdown
import json
import urllib
import string
import random
import copy
from github import Github, Auth
from secret import GH_CREDS
from pprint import pprint


In [5]:
#add path to workflow
sys.path.append("../scripts/") 
#test import
from crosswalk_mappings import *

## Access the original submission (issue) based on number

In [11]:
auth = Auth.Token(GH_CREDS)
g = Github(auth=auth)
g.get_user().login
repo = g.get_repo('ModelAtlasofTheEarth/Model_Submission')

In [12]:
# get issue based on issue number (https://github.com/ModelAtlasofTheEarth/model_submission/issues)
issue_number = 26
issue = repo.get_issue(number = issue_number)

## Manually extract the issue.body into a dictionary

This represents the "first level" parsing, 

* converts the issue Markdown body into a Python dictionary.
* Each section heading in the GitHub Model Submission template becomes a dictionary key
* the user’s response under each heading becomes the corresponding dictionary value.


In [24]:
## note that in the Github workflow - this bit happens within 'parse_issue'

regex = r"### *(?P<key>.*?)\s*[\r\n]+(?P<value>[\s\S]*?)(?=###|$)"
data = dict(re.findall(regex, issue.body))

In [25]:
data['-> submitter ORCID (or name)']

'0000-0002-2207-6837\r\n\r\n'

In [19]:
#issue.body

## Parse issue 

ParseIssue Function – Workflow Summary

- `ParseIssue` is the second processing layer and performs the main “heavy lifting.”
- It acts as a wrapper/orchestrator for many subordinate functions that structure and normalise the issue data.
- Input: the dictionary produced from the first step (where GitHub template headings become keys).
- Output: a set of structured sub-dictionaries that will later be mapped into a formal JSON metadata document (e.g., an RO-Crate).

- `ParseIssue` calls a range of helper functions to:
  - clean and normalise fields
  - split complex responses into nested structures
  - validate identifiers
  - prepare fields for downstream conversion into RO-Crate entities

- A key part of this step involves identifier resolution.
  - Functions like `GetRecord` are used to retrieve metadata from external sources.
  - `GetRecord` is generic but must know the record type (e.g., DOI, ORCID, ROR, funder IDs).
  - Depending on the type, it dispatches to record-specific parsers such as:
    - `ParseOrganization`
    - `ProcessFundingData`
    - `ParsePerson`
    - etc.

- These functions may make external API calls (e.g., DOI APIs, ORCID, Crossref, ROR).
- API responses then need to be parsed, normalised, and converted into internal structures compatible with the final metadata schema.

Overall: `ParseIssue` is the orchestration layer that turns a loosely structured human-written GitHub issue into a fully structured internal metadata model, ready for conversion into RO-Crate JSON. 


In [21]:
from parse_issue import parse_issue 


In [22]:
issue_dict, log = parse_issue(issue)

https://api.crossref.org/works/https://doi.org/10.1093/gji/ggad230
https://doi.org/10.5281/zenodo.6903424
https://pub.orcid.org/v3.0/0000-0003-2311-9402
https://pub.orcid.org/v3.0/0000-0003-0357-7115
https://pub.orcid.org/v3.0/0000-0003-0035-7723
https://pub.orcid.org/v3.0/0000-0001-7098-8198
https://pub.orcid.org/v3.0/0000-0002-9481-1749
https://pub.orcid.org/v3.0/0000-0002-8137-3903
https://pub.orcid.org/v3.0/0000-0001-9489-5236
https://pub.orcid.org/v3.0/0000-0002-5697-7203
https://pub.orcid.org/v3.0/0000-0002-2207-6837
https://api.ror.org/organizations/05mmh0f86
https://api.ror.org/organizations/03wnrjx87
https://api.ror.org/organizations/02b5d8509
https://doi.org/https://dx.doi.org/10.25914/608bfd1838db2


Check what's in "issue_dict"

In [26]:
issue_dict['publication']

{'@type': 'ScholarlyArticle',
 '@id': 'https://doi.org/10.1093/gji/ggad230',
 'name': 'Plate bending earthquakes and the strength distribution of the lithosphere',
 'isPartOf': ({'@type': 'PublicationIssue',
   'issueNumber': '1',
   'datePublished': '2023-5-27',
   'isPartOf': {'@type': ['PublicationVolume', 'Periodical'],
    'name': ['Geophysical Journal International'],
    'issn': ['0956-540X', '1365-246X'],
    'volumeNumber': '235',
    'publisher': 'Oxford University Press (OUP)'}},),
 'author': [{'@type': 'Person',
   '@id': 'https://orcid.org/0000-0002-2207-6837',
   'givenName': 'Dan',
   'familyName': 'Sandiford',
   'affiliation': [{'@type': 'Organization',
     'name': 'School of Earth, Atmosphere and Environment, Monash University , Melbourne, VIC 3800, Australia (formerly)'}]},
  {'@type': 'Person',
   '@id': 'https://orcid.org/0000-0003-2198-9172',
   'givenName': 'Timothy J',
   'familyName': 'Craig',
   'affiliation': [{'@type': 'Organization',
     'name': 'COMET, I

In [27]:
len(issue_dict['creators'])

2

## Testing the subordinate functions

In [32]:
from request_utils import get_record
from parse_metadata_utils import  parse_organization
from parse_utils import process_funding_data, get_funders, parse_organization

In [29]:
print(data["-> funder"])
funders_string = data["-> funder"]
funders_dict = process_funding_data(funders_string)
print(funders_dict)

https://ror.org/05mmh0f86, DP150102887
https://ror.org/03wnrjx87, URF\R1\180088
https://ror.org/02b5d8509


https://api.ror.org/organizations/05mmh0f86
https://api.ror.org/organizations/03wnrjx87
https://api.ror.org/organizations/02b5d8509
{'funders': [{'@type': 'Organization', '@id': 'https://ror.org/05mmh0f86', 'name': 'ARC', 'url': 'http://www.arc.gov.au/', 'additionalType': ['funder', 'government']}, {'@type': 'Organization', '@id': 'https://ror.org/03wnrjx87', 'name': 'Royal Society', 'url': 'https://royalsociety.org/', 'additionalType': ['funder', 'nonprofit']}, {'@type': 'Organization', '@id': 'https://ror.org/02b5d8509', 'name': 'NERC', 'url': 'https://www.ukri.org/councils/nerc/', 'additionalType': ['funder', 'government']}], 'funding': [{'@type': 'Grant', 'funder': {'@type': 'Organization', '@id': 'https://ror.org/05mmh0f86', 'name': 'ARC', 'url': 'http://www.arc.gov.au/', 'additionalType': ['funder', 'government']}, 'identifier': 'DP150102887'}, {'@type': 'Grant', 'funder': 

# Test get_record/parse_organization  directly 

* Here we are testing the get_record by handing in a ROR id
* then using parse_organization to handle that record

In [36]:
# Test the underlying API directly
print("\n🔹 Testing get_record() directly on one ROR ID...")
record, log = get_record("organization", "05mmh0f86")
print("Record keys:", list(record.keys()))
print("Log:", log)

# Test parsing that record into schema.org format
print("\n🔹 Testing parse_organization()...")
parsed, parse_log = parse_organization(record)
pprint(parsed)
print("Parse log:", parse_log)


🔹 Testing get_record() directly on one ROR ID...
https://api.ror.org/organizations/05mmh0f86
Record keys: ['admin', 'domains', 'established', 'external_ids', 'id', 'links', 'locations', 'names', 'relationships', 'status', 'types']
Log: 

🔹 Testing parse_organization()...
{'@id': 'https://ror.org/05mmh0f86',
 '@type': 'Organization',
 'additionalType': ['funder', 'government'],
 'name': 'ARC',
 'url': 'http://www.arc.gov.au/'}
Parse log: 


# Converting the issue_dict

In [50]:
from crosswalks import dict_to_report, dict_to_metadata
from ro_crate_utils import load_entity_template, recursively_filter_key
from crosswalks import dict_to_report, dict_to_metadata, dict_to_ro_crate_mapping, default_issue_entity_mapping_list
from ro_crate_utils import load_crate_template, defaults_and_customise_ro_crate


## Issue to report

In [42]:

dict_to_metadata = dict_to_report(issue_dict)

## Issue to metadata

In [45]:
# Perform a deep copy of the issue_dict to avoid modifying the input dictionary
issue_dict_copy = copy.deepcopy(issue_dict)

#this takes the issue_dict and simplifies entities (e.g. @Type=Person) using templates defined at:
#https://github.com/ModelAtlasofTheEarth/metadata_schema/blob/main/mate_ro_crate/type_templates.json

entity_template = load_entity_template()
recursively_filter_key(issue_dict_copy, entity_template)

#this maps the issue_dict_copy to the ro_crate
rocrate = dict_to_metadata(issue_dict_copy, flat_compact_crate=False)

JSON-LD data loaded successfully.


In [64]:
type(rocrate)

str

In [65]:
#dict_to_metadata?

## Further notes

`dict_to_metadata` loads a loads the base RO-Crate skeleton, containing:

* @context
* top-level @graph with placeholders
* default MATE-specific RO-Crate metadata

- the mapping_list (list) argument of dict_to_metadata: A list of mappings that define how elements in the issue dictionary correspond to elements in the RO-Crate structure.

In [66]:
#dict_to_metadata produces a string, suitable for writign the rocrate to file. 
# we can also load this a dictionary

rocratedict = json.loads(rocrate)

rocratedict['@graph'][1]['citation']

{'@type': 'ScholarlyArticle',
 '@id': 'https://doi.org/10.1093/gji/ggad230',
 'name': 'Plate bending earthquakes and the strength distribution of the lithosphere',
 'author': [{'@type': 'Person',
   '@id': 'https://orcid.org/0000-0002-2207-6837',
   'givenName': 'Dan',
   'familyName': 'Sandiford'},
  {'@type': 'Person',
   '@id': 'https://orcid.org/0000-0003-2198-9172',
   'givenName': 'Timothy J',
   'familyName': 'Craig'}],
 'abstract': 'This study investigates the dynamics and constitutive behaviour of the oceanic lithosphere as it bends and yields during subduction. Two main observational constraints are considered: the maximum bending moment that can be supported by the lithosphere, and the inferred neutral plane depth in bending. We particularly focus on regions of old lithosphere where the ‘apparent’ neutral plane depth is about 30\xa0km. We use subduction modelling approaches to investigate these flexural characteristics. We reassess bending moment estimates from a range of pr